In [ ]:
import os
import random
import time

from test_write import PlainWrite
from test_flush import Flush
from test_append import Append


SIZES = [1000000, 2000000, 3000000, 5000000, 10000000]
TYPES = ["text", "binary"]
INTER_FILE = [0, 0.5, 1, 2, 3, 5]
CHUNK_SIZE = [100000, 200000, 300000, 500000]
INTER_CHUNK = [0, 0.1, 0.2, 0.3, 0.5, 1]

CHECK_DATA = True

TEST = 10
CYCLE = 60

# This is just to run tests forever from the cboxtu account
OUTPUT_FOLDER = "/eos/user/c/cboxtu/Enrico_swanIOtesting"

# Print a legend line for metadata
print " ".join(["file_numer", "file_size", "file_type", "inter_file", "chunk_size", "inter_chunk"])

# Run the tests forever
i=0
while (True):
    i += 1
    print "Test no. %d\n" % i

    # Pick the values you like and print them out
    fno = random.choice(range(10))+1          # File number
    fsi = random.choice(SIZES)                # File size
    fty = random.choice(TYPES)                # File types

    ifi = random.choice(INTER_FILE)           # Wait time between write of two files

    csi = (random.choice(range(5))+1)*100000  # Chunk size (for flush and append tests)
    ich = random.choice(INTER_CHUNK)          # Wait time between write of two chunks (for flush and append tests)

    print fno, fsi, fty, ifi, csi, ich

    # Do the tests
    print "PlainWrite"
    #def __init__(self, fno, fsize, ftype, itime=0, sanity_check=False, output_folder=CWD)
    current_test = PlainWrite(fno, fsi, fty, ifi, True, OUTPUT_FOLDER)
    current_test.run()
    time.sleep(TEST)

    print "Flush"
    #def __init__(self, fno, fsize, ftype, flush_size=4096, flush_time=0, itime=0, sanity_check=False, output_folder=CWD)
    current_test = Flush(fno, fsi, fty, csi, ich, ifi, True, OUTPUT_FOLDER)
    current_test.run()
    time.sleep(TEST)

    print "Append"
    #def __init__(self, fno, fsize, ftype, chunk_size=4096, chunk_time=0, itime=0, sanity_check=False, output_folder=CWD)
    current_test = Append(fno, fsi, fty, csi, ich, ifi, True, OUTPUT_FOLDER)
    current_test.run()
    time.sleep(TEST)

    time.sleep(CYCLE)
